<h2>3D stack to 2D MIP - Batch processing - Marker+ based on average intensity</h2>

The following notebook is able to process a 3D stack (.czi or .nd2 files) into a MIP and using the parameters tested in <code>000_SP_Avg_Intensity</code> allows you to:

1. Read previously defined ROIs, if not present, full image is analyzed.
2. Read previously predicted nuclei labels, if not present, generates them.
3. Extract numbers of cells positive for all marker based on signal average intensity within the nuclear or cytoplasmic compartments (using a user-defined min-max range).
4. Extract and save per label per ROI per marker data in a .csv file (avg_int_per_label.csv).
5. Extract and save number and % of positive cells in a .csv file (BP_marker_+_label_avg_int_2D.csv).

In [1]:
from pathlib import Path
import tifffile
import napari
import os
from tqdm import tqdm
import numpy as np
import pandas as pd
from skimage.measure import regionprops_table
import plotly.express as px
from utils import list_images, read_image, simulate_cytoplasm, segment_nuclei_2d

<h3>Define the directory for your images (.nd2 or .czi files) and cell marker info</h3>

In [2]:
# Copy the path where your images are stored, ideally inside the raw_data directory
directory_path = Path("./raw_data/test_data")

# Define the channels you want to analyze using the following structure:
# markers = [(channel_name, channel_nr, cellular_location),(..., ..., ...)]
# Remember in Python one starts counting from 0, so your first channel will be 0
markers = [("ki67", 0, "nucleus"), ("neun", 1, "nucleus"), ("calbindin", 2, "cytoplasm")]

# Iterate through the .czi and .nd2 files in the raw_data directory
images = list_images(directory_path)

images

['raw_data\\test_data\\HI 1  Contralateral Mouse 8  slide 6 Neun Red Calb Green KI67 Magenta 40x technical replica 1.czi',
 'raw_data\\test_data\\HI 1  Ipsilateral Mouse 8  slide 6 Neun Red Calb Green KI67 Magenta 40x technical replica 1.czi']

<h3>Define you batch analysis parameters</h3>

If you have generated nuclei predictions already, make sure to input the same <code>slicing factor</code> you used when generating nuclei predictions. 

If you have not generated nuclei predictions before, input <code>nuclei_channel</code>, <code>cellpose_nuclei_diameter</code> and <code>gaussian_sigma</code> values.

Define in <code>min_max_per_marker</code> the <code>marker</code> you want to use to define your cell populations of interest, the <code>min_max</code> range of avg_int and the <code>population</code> name.

In [3]:
# Remember to input the same slicing factor you used when generating the nuclei predictions
slicing_factor = None # Use 2 or 4 for compression (None for lossless)

# Define the nuclei and markers of interest channel order ('Remember in Python one starts counting from zero')
nuclei_channel = 3

# Define your nuclei diameter, it speeds up nuclei detection, if unknown leave it as None
cellpose_nuclei_diameter = None

# Define the amount of blur applied to nuclei
# Blurs the mip_nuclei image to even out high intensity foci within the nucleus, the higher the value blurriness increases
# High values help segment sparse nuclei (CA and CTX regions) but as a drawback it merges nuclei entities that are very close together (DG region)
gaussian_sigma = 0 

# Define the min_max average intensity parameters to select your populations of interest
# You have the possibility to define populations for the same marker (i.e. neun high and neun low)
# max_values are set to 255 since the test input images are 8-bit, higher bit depths can result in higher max avg_int values

min_max_per_marker = [{"marker": "ki67", "min_max": (200,255), "population":"ki67"},
                      {"marker": "neun", "min_max": (50,115), "population":"neun_low"},
                      {"marker": "neun", "min_max": (115,255), "population":"neun_high"},
                      {"marker": "calbindin", "min_max": (65,255), "population":"calbindin"}]

<h3>Run Batch Analysis</h3>

In [4]:
# Construct ROI and nuclei predictions paths from directory_path above
roi_path = directory_path / "ROIs"
nuclei_preds_path =  directory_path / "nuclei_preds"

# Extract the experiment name from the data directory path
experiment_id = directory_path.name

# List of subfolder names
try:
    roi_names = [folder.name for folder in roi_path.iterdir() if folder.is_dir()]

except FileNotFoundError:
    roi_names = ["full_image"]
        
print(f"The following regions of interest will be analyzed: {roi_names}")

for image in tqdm(images):

    # Generate maximum intensity projection and extract filename
    img_mip, filename = read_image(image, slicing_factor)

    # Initialize an empty list to hold the extracted dataframes on a per ROI basis
    per_roi_props = []

    for roi_name in roi_names:

        # Initialize an empty list to hold the extracted dataframes on a per channel basis
        props_list = []

        # Read the user defined ROIs, in case of full image analysis generate a label covering the entire image
        try:
            # Read previously defined ROIs
            user_roi = tifffile.imread(roi_path / roi_name / f"{filename}.tiff")

        except FileNotFoundError:
            # Extract the xy dimensions of the input image 
            img_shape = img_mip.shape
            img_xy_dims = img_shape[-2:]

            # Create a label covering the entire image
            user_roi = np.ones(img_xy_dims).astype(np.uint8)

        # Read previously predicted nuclei labels, if not present generate nuclei predictions and save them
        try:
            # Read the nuclei predictions per ROI
            nuclei_labels = tifffile.imread(nuclei_preds_path / roi_name / f"{filename}.tiff")

        except FileNotFoundError:

            # Slice the nuclei stack
            nuclei_img = img_mip[nuclei_channel, :, :]

            # We will create a mask where roi is greater than or equal to 1
            mask = (user_roi >= 1).astype(np.uint8)

            # Apply the mask to nuclei_img and marker_img, setting all other pixels to 0
            masked_nuclei_img = np.where(mask, nuclei_img, 0)

            # Segment nuclei and return labels
            nuclei_labels = segment_nuclei_2d(masked_nuclei_img, gaussian_sigma, cellpose_nuclei_diameter)

            # Save nuclei labels as .tiff files to reuse them later
            # Create nuclei_predictions directory if it does not exist
            try:
                os.makedirs(directory_path / "nuclei_preds" / roi_name)
            except FileExistsError:
                pass

            # Construct path to store
            path_to_store = directory_path / "nuclei_preds" / roi_name / f"{filename}.tiff"

            # Save mask (binary image)
            tifffile.imwrite(path_to_store, nuclei_labels)

        # Create a dictionary containing all image descriptors
        descriptor_dict = {
                    "filename": filename,
                    "ROI": roi_name,
                    }

        # Loop through each channel and extract the average intensity within either nuclei or cytoplasmic regions
        for tuple in markers:

            channel_name = tuple[0]
            ch_nr = tuple[1]
            location = tuple[2]

            if location == "cytoplasm":

                # Simulate a cytoplasm by dilating the nuclei and substracting the nuclei mask afterwards
                cytoplasm_labels = simulate_cytoplasm(nuclei_labels, dilation_radius = 2, erosion_radius = 0)

                # Extract intensity information from each marker channel
                props = regionprops_table(label_image=cytoplasm_labels,
                                    intensity_image=img_mip[ch_nr],
                                    properties=["label", "intensity_mean"])
            
            elif location == "nucleus":

                # Extract intensity information from each marker channel
                props = regionprops_table(label_image=nuclei_labels,
                                    intensity_image=img_mip[ch_nr],
                                    properties=["label", "intensity_mean"])
            

            # Convert to dataframe
            props_df = pd.DataFrame(props)

            # Rename intensity_mean column to indicate the specific image
            props_df.rename(columns={"intensity_mean": f"{location}_{channel_name}_avg_int"}, inplace=True)

            # Append each props_df to props_list
            props_list.append(props_df)

            # Initialize the df with the first df in the list
            props_df = props_list[0]
            # Start looping from the second df in the list
            for df in props_list[1:]:
                props_df = props_df.merge(df, on="label")

        # Add each key-value pair from descriptor_dict to props_df at the specified position
        insertion_position = 0    
        for key, value in descriptor_dict.items():
            props_df.insert(insertion_position, key, value)
            insertion_position += 1  # Increment position to maintain the order of keys in descriptor_dict

        # Append each props_df to props_list
        per_roi_props.append(props_df)

    final_df = pd.concat(per_roi_props, ignore_index=True)

    # Create a 'results' folder in the root directory
    results_folder = Path("results") / experiment_id

    try:
        os.makedirs(results_folder)
        print(f"'{results_folder}' folder created successfully.")
    except FileExistsError:
        print(f"'{results_folder}' folder already exists.")

    # Save the df containing per_label results into a CSV file
    final_df.to_csv(results_folder / f'{filename}_per_label.csv')

    # Select all column names in 'final_df' that contain the substring 'avg_int'
    avg_int_columns = [col for col in final_df.columns if 'avg_int' in col]

    # Create an empty list to store all stats extracted from each image
    stats = []

    for marker_analysis in min_max_per_marker:

        marker = marker_analysis["marker"]
        min_max_avg_int = marker_analysis["min_max"]
        population = marker_analysis["population"]

        # Retrieve the column name from which the avg_int values should be read
        for column in avg_int_columns:
            if marker in column:
                column_name = column

        for roi_name in roi_names:

            # Initialize an empty list to hold the extracted dataframes on a per channel basis
            props_list = []

            # Retrieve the first and second values (channel and location) of the corresponding tuple in markers
            for item in markers:
                if item[0] == marker:
                    channel = item[1]
                    location = item[2]
                    break  # Stop searching once the marker is found

            # Read the nuclei predictions per ROI
            nuclei_labels = tifffile.imread(nuclei_preds_path / roi_name / f"{filename}.tiff")

            # Filter rows in final_df where ROI matches roi_name and column_name is higher than threshold
            filtered_df = final_df[(final_df["ROI"] == roi_name) & (final_df[column_name] > min_max_avg_int[0]) & (final_df[column_name] < min_max_avg_int[1])]

            # Get the values of the 'label' column in filtered_df as a list
            label_values = filtered_df["label"].tolist()

            # Create a boolean mask where each element is True if the corresponding value in 'nuclei_labels' 
            # is found in 'label_values', and False otherwise
            mask = np.isin(nuclei_labels, label_values)

            # Use the mask to set values in 'nuclei_labels' that are not in 'label_values' to 0,
            # creating a new array 'filtered_labels' with only the specified values retained
            filtered_labels = np.where(mask, nuclei_labels, 0)

            # Extract your information of interest
            total_nuclei = len(np.unique(nuclei_labels)) - 1
            marker_pos_nuclei = len(np.unique(filtered_labels)) - 1

            # Calculate "%_marker+_cells" and avoid division by zero errors
            try:
                perc_marker_pos_cells = (marker_pos_nuclei * 100) / total_nuclei
            except ZeroDivisionError:
                perc_marker_pos_cells = 0

            # Create a dictionary containing all extracted info per masked image
            stats_dict = {
                        "filename": filename,
                        "ROI": roi_name,
                        "based_on": column_name,
                        "marker": marker,
                        "population":population,
                        "marker_ch": channel,
                        "location": location,
                        "min_max_avg_int": min_max_avg_int,
                        "total_nuclei": total_nuclei,
                        "marker+_nuclei": marker_pos_nuclei,
                        "%_marker+_cells": perc_marker_pos_cells,
                        "slicing_factor": slicing_factor
                        }
            
            # Append the current data point to the stats_list
            stats.append(stats_dict)  

    # Create the necessary folder structure if it does not exist
    try:
        os.makedirs(str(results_folder))
        print(f"Output folder created: {results_folder}")
    except FileExistsError:
        print(f"Output folder already exists: {results_folder}")

    # Transform into a dataframe to store it as .csv later
    df = pd.DataFrame(stats)

    # Define the .csv path
    csv_path = results_folder / "BP_marker_+_label_avg_int_2D.csv"

    # Append to the .csv with new data points each round
    df.to_csv(csv_path, mode="a", index=False, header=not os.path.isfile(csv_path))

# Show the updated .csv 
csv_df = pd.read_csv(csv_path)

csv_df 

The following regions of interest will be analyzed: ['ALL', 'CA', 'DG']


  0%|          | 0/2 [00:00<?, ?it/s]

Image analyzed: HI 1  Contralateral Mouse 8  slide 6 Neun Red Calb Green KI67 Magenta 40x technical replica 1
Original Array shape: (4, 14, 3803, 2891)
MIP Array shape: (4, 3803, 2891)
'results\test_data' folder created successfully.


 50%|█████     | 1/2 [01:16<01:16, 76.93s/it]

Output folder already exists: results\test_data
Image analyzed: HI 1  Ipsilateral Mouse 8  slide 6 Neun Red Calb Green KI67 Magenta 40x technical replica 1
Original Array shape: (4, 12, 3798, 2877)
MIP Array shape: (4, 3798, 2877)
'results\test_data' folder already exists.


100%|██████████| 2/2 [02:28<00:00, 74.39s/it]

Output folder already exists: results\test_data


,filename,ROI,based_on,marker,population,marker_ch,location,min_max_avg_int,total_nuclei,marker+_nuclei,%_marker+_cells,slicing_factor
0,HI 1 Contralateral Mouse 8 slide 6 Neun Red ...,ALL,nucleus_ki67_avg_int,ki67,ki67,0,nucleus,"(200, 255)",5054,226,4.471706,NaN
1,HI 1 Contralateral Mouse 8 slide 6 Neun Red ...,CA,nucleus_ki67_avg_int,ki67,ki67,0,nucleus,"(200, 255)",1223,7,0.572363,NaN
2,HI 1 Contralateral Mouse 8 slide 6 Neun Red ...,DG,nucleus_ki67_avg_int,ki67,ki67,0,nucleus,"(200, 255)",1912,107,5.596234,NaN
3,HI 1 Contralateral Mouse 8 slide 6 Neun Red ...,ALL,nucleus_neun_avg_int,neun,neun_low,1,nucleus,"(50, 115)",5054,1307,25.860704,NaN
4,HI 1 Contralateral Mouse 8 slide 6 Neun Red ...,CA,nucleus_neun_avg_int,neun,neun_low,1,nucleus,"(50, 115)",1223,208,17.007359,NaN
5,HI 1 Contralateral Mouse 8 slide 6 Neun Red ...,DG,nucleus_neun_avg_int,neun,neun_low,1,nucleus,"(50, 115)",1912,951,49.738494,NaN
6,HI 1 Contralateral Mouse 8 slide 6 Neun Red ...,ALL,nucleus_neun_avg_int,neun,neun_high,1,nucleus,"(115, 255)",5054,1230,24.337159,NaN
7,HI 1 Contralateral Mouse 8 slide 6 Neun Red ...,CA,nucleus_neun_avg_int,neun,neun_high,1,nucleus,"(115, 255)",1223,852,69.664759,NaN
8,HI 1 Contralateral Mouse 8 slide 6 Neun Red ...,DG,nucleus_neun_avg_int,neun,neun_high,1,nucleus,"(115, 255)",1912,372,19.456067,NaN
9,HI 1 Contralateral Mouse 8 slide 6 Neun Red ...,ALL,cytoplasm_calbindin_avg_int,calbindin,calbindin,2,cytoplasm,"(65, 255)",5054,90,1.780768,NaN
